## <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone

from features.interactions import month_sine, month_cosine

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## <span style='color:#ff5f27'> 🔗 Fetch interactions feature group </span>

In [ ]:
interactions_fg = fs.get_feature_group(
    name="interactions",
    version=1)
    
data_interactions_df = interactions_fg.read()
data_interactions_df

In [ ]:
data_interactions_df = data_interactions_df[["id", "interaction_date", "interaction_month"]]
# Calculate the sine and cosine components for the month_of_purchase
data_interactions_df["month_sin"] = data_interactions_df.interaction_date.map(lambda x: month_sine(x))
data_interactions_df["month_cos"] = data_interactions_df.interaction_date.map(lambda x: month_cosine(x)) 
data_interactions_df         

## <span style="color:#ff5f27">🪄 Feature Group Creation </span>

In [ ]:
interactions_month_sincos_fg = fs.get_or_create_feature_group(
    name="interactions_month_sincos",
    version=1,
    description="Ondeamand Features for Interactions data such month sine and cosine.",    
    primary_key=["id"],
    partition_key = ["interaction_month"],
    online_enabled=True,
    event_time="interaction_date",
    parents=[interactions_fg],
    statistics_config = {
        "enabled": True,
        "histograms": True,
        "correlations": True,
    }    
)

interactions_month_sincos_fg.insert(data_interactions_df)
print('Done ✅')

In [ ]:
feature_descriptions = [
    {"name": "id", "description": "Unique id for the interaction"},
    {"name": "month_sin", "description": "Sine of the month of interaction date."},
    {"name": "month_cos", "description": "Cosine of the month of interaction date."},    
    {"name": "interaction_date", "description": "Date of inteaction."},
    {"name": "interaction_month", "description": "Month of interaction, derived from interaction_date."}
]

for desc in feature_descriptions:
    interactions_month_sincos_fg.update_feature_description(desc["name"], desc["description"])

---